### Import Libraries

In [5]:
import os
import csv
import json
import openai

from dotenv import load_dotenv

### Load Environment Variables

In [7]:
load_dotenv()

api_key = os.environ.get("API_KEY")
org_key = os.environ.get("ORG_KEY")

### Load CSV Rephrase Data

In [ ]:
import csv

# Specify the path to your CSV file
csv_file_path = 'your_file.csv'

# Initialize an empty list to store the data
data_list = []

# Open the CSV file for reading
with open(csv_file_path, newline='') as csvfile:
    # Create a CSV reader object
    csv_reader = csv.DictReader(csvfile)
    
    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Append the row (as a dictionary) to the data_list
        data_list.append(row)

# Now, data_list contains a list of dictionaries where each dictionary represents a row of data
# The keys in each dictionary are taken from the CSV header

# You can access the data like this:
for item in data_list:
    print(item['attribute1'])  # Replace 'attribute1' with the actual attribute name
    print(item['attribute2'])  # Replace 'attribute2' with the actual attribute name
    # ...

# You can access the data in any way you like using the keys from the header


In [ ]:
# Define your OpenAI API key
api_key = 'YOUR_API_KEY_HERE'

# Define a function to generate prompts based on the conditions
def generate_prompt(row):
    if row['name'] == 'True' and row['concept'] == 'False' and row['option'] == 'False':
        return f"Change all names in the given phrases to Indonesian names. Change only the name and keep the remaining phrases in English. Reply with only your answer, do not repeat the given phrase.\n\nPhrase: {row['question']}\nAnswer:"
    else:
        return f"Given a commonsense question, a concept, options, and the question answer, change them to become relevant to Indonesia. If an aspect is flagged to be changed, then you need to change it completely. If it's flagged as keep, then keep as it is. Make sure your changes are still in the same domain/topic with the given data, and there is only one clear answer in the options. Reply with only your changed data in a JSON format.\n\nData:\n###\nQuestion: {row['question']} -> Change\nConcept: {row['question_concept']} -> {'Keep' if row['concept'] == 'True' else 'Change'}\nOptions: -> {'Keep' if row['option'] == 'True' else 'Change'}\n{row['choices']}Question Answer: {row['answerKey']} + {row['choices']}###\n\nChanged data:"

# Define a function to rephrase the CSV data using OpenAI GPT-3.5-Turbo
def rephrase_csv_data(input_csv, api_key):
    openai.api_key = api_key

    with open(input_csv, 'r') as file:
        lines = file.readlines()

    # Remove the header line
    header = lines[0]
    lines = lines[1:]

    rephrased_data = []

    for line in lines:
        parts = line.strip().split(',')
        if len(parts) == 7:
            row = {
                'question': parts[0],
                'question_concept': parts[1],
                'choices': parts[2],
                'answerKey': parts[3],
                'concept': parts[4],
                'name': parts[5],
                'option': parts[6]
            }

            prompt = generate_prompt(row)

            response = openai.Completion.create(
                engine="text-davinci-002",  # You can also use "davinci" instead of "text-davinci-002"
                prompt=prompt,
                max_tokens=100  # Adjust as needed
            )

            rephrased_data.append({
                'original_data': row,
                'rephrased_response': response.choices[0].text.strip()
            })

    return rephrased_data

# Specify the path to your input CSV file
input_csv_file = 'input.csv'

# Call the function to rephrase the CSV data
rephrased_data = rephrase_csv_data(input_csv_file, api_key)

# Print or save the rephrased data as needed
for item in rephrased_data:
    print(f'Original Data: {item["original_data"]}')
    print(f'Rephrased Response: {item["rephrased_response"]}')
    print('\n')

# Optionally, you can save the rephrased data to a JSON file
with open('rephrased_data.json', 'w') as json_file:
    json.dump(rephrased_data, json_file, indent=4)
